# Codage des caractères



Points traités dans ce notebook :

- Mettre en place notre environnement
- Que sont les encodages ?
- Lire des fichiers avec des problèmes d'encodage
- Sauvegarder vos fichiers avec l'encodage UTF-8

# Mise en place de notre environnement

La première chose à faire est de charger les bibliothèques que nous utiliserons. Mais pas nos jeux de données : nous y reviendrons plus tard !

Important ! Assurez-vous d'exécuter cette cellule vous-même ou le reste de votre code ne fonctionnera pas !

In [25]:
# librairies
import pandas as pd
import numpy as np

# librairie d'encodage de caractères utile
import chardet

# reproductibilité
np.random.seed(0)

# Qu'est-ce qu'un encodage ?

Les codages de caractères sont des ensembles de règles spécifiques permettant de passer de chaînes d'octets binaires brutes (qui ressemblent à ceci : 0110100001101001) aux caractères qui composent un texte lisible par l'homme (comme "hi"). Il existe de nombreux codages différents, et si vous essayez de lire un texte avec un codage différent de celui dans lequel il a été écrit à l'origine, vous obtenez un texte brouillé appelé "mojibake" (qui se prononce comme mo-gee-bah-kay). Voici un exemple de mojibake :

æ-‡å-åŒ-ã ??

Vous pouvez également vous retrouver avec des caractères "inconnus". C'est ce qui s'affiche lorsqu'il n'y a pas de correspondance entre un octet particulier et un caractère dans l'encodage que vous utilisez pour lire votre chaîne d'octets et qui ressemble à ceci :

����������

Les erreurs d'encodage de caractères sont moins fréquentes aujourd'hui qu'elles ne l'étaient auparavant, mais elles constituent toujours un problème. Il existe de nombreux codages de caractères différents, mais le principal que vous devez connaître est UTF-8.

UTF-8 est l'encodage de texte standard. Tout le code Python est en UTF-8 et, idéalement, toutes vos données devraient l'être aussi. C'est lorsque les choses ne sont pas en UTF-8 que vous rencontrez des problèmes.

Il était assez difficile de gérer les encodages dans Python 2, mais heureusement, dans Python 3, c'est beaucoup plus simple. (Les Kaggle Kernels n'utilisent que Python 3.) Il y a deux types de données principaux que vous rencontrerez lorsque vous travaillerez avec du texte en Python 3. Le premier est la chaîne de caractères, qui est le type de texte par défaut.

Nous sommes maintenant prêts à travailler avec des encodages de caractères ! (Si vous le souhaitez, vous pouvez ajouter une cellule de code ici et profiter de l'occasion pour jeter un coup d'œil à certaines données).

In [26]:
# commencer par une chaîne de caractères
before = "This is the euro symbol: €"

# vérifie quel est le type de données
type(before)

str

L'autre donnée est le type de données bytes, qui est une séquence d'entiers. Vous pouvez convertir une chaîne de caractères en octets en spécifiant l'encodage utilisé :

In [27]:
#  l'encoder dans un encodage différent, en remplaçant les caractères qui soulèvent des erreurs
after = before.encode("utf-8", errors = "replace")

# vérifier le type
type(after)

bytes

Si vous regardez un objet octet, vous verrez qu'il est précédé d'un b et qu'il est suivi d'un peu de texte. C'est parce que les octets sont imprimés comme s'ils étaient des caractères codés en ASCII. (L'ASCII est un ancien codage de caractères qui ne fonctionne pas vraiment pour l'écriture d'une langue autre que l'anglais). Ici, vous pouvez voir que notre symbole de l'euro a été remplacé par un mojibake qui ressemble à "\xe2\x82\xac" lorsqu'il est imprimé comme s'il s'agissait d'une chaîne ASCII.

In [28]:
# voir à quoi ressemblent les octets
after

b'This is the euro symbol: \xe2\x82\xac'

Lorsque nous reconvertissons nos octets en une chaîne de caractères avec le bon encodage, nous pouvons voir que notre texte est bien présent, ce qui est très bien !)

In [29]:
# le reconvertir en utf-8
print(after.decode("utf-8"))

This is the euro symbol: €


Cependant, lorsque nous essayons d'utiliser un encodage différent pour convertir nos octets en une chaîne de caractères, nous obtenons une erreur. C'est parce que l'encodage que nous essayons d'utiliser ne sait pas quoi faire avec les octets que nous essayons de lui passer. Vous devez indiquer à Python l'encodage dans lequel la chaîne d'octets est censée être.

On peut comparer les différents encodages à différentes manières d'enregistrer de la musique. Vous pouvez enregistrer la même musique sur un CD, une cassette ou un 8 pistes. Bien que la musique ait plus ou moins le même son, vous devez utiliser l'équipement approprié pour lire la musique de chaque format d'enregistrement. Le décodeur approprié est comme un lecteur de cassettes ou un lecteur de CD. Si vous essayez de lire une cassette dans un lecteur de CD, cela ne fonctionnera pas.

In [30]:
# essayer de décoder nos octets avec l'encodage ascii
print(after.decode("ascii"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 25: ordinal not in range(128)

Nous pouvons également rencontrer des problèmes si nous essayons d'utiliser le mauvais encodage pour passer d'une chaîne à des octets. Comme je l'ai dit plus haut, les chaînes sont UTF-8 par défaut dans Python 3, donc si nous essayons de les traiter comme si elles étaient dans un autre encodage, nous allons créer des problèmes.

Par exemple, si nous essayons de convertir une chaîne en octets pour l'ascii en utilisant encode(), nous pouvons demander que les octets soient ce qu'ils seraient si le texte était en ASCII. Mais comme notre texte n'est pas en ASCII, il y aura des caractères qu'il ne pourra pas gérer. Nous pouvons remplacer automatiquement les caractères que l'ASCII ne peut pas gérer. Si nous faisons cela, cependant, tous les caractères qui ne sont pas en ASCII seront simplement remplacés par le caractère inconnu. Ensuite, lorsque nous reconvertissons les octets en chaîne de caractères, le caractère sera remplacé par le caractère inconnu. Ce qui est dangereux, c'est qu'il n'y a aucun moyen de savoir quel caractère aurait dû être remplacé. Cela signifie que nous venons peut-être de rendre nos données inutilisables !

In [ ]:
# commencer par une chaîne de caractères
before = "This is the euro symbol: €"

# l'encoder dans un encodage différent, en remplaçant les caractères qui soulèvent des erreurs
after = before.encode("ascii", errors = "replace")

# le reconvertir en utf-8
print(after.decode("ascii"))

# Nous avons perdu la chaîne d'octets sous-jacente d'origine ! Elle a été 
# remplacée par la chaîne d'octets sous-jacente du caractère inconnu :(

This is the euro symbol: ?


C'est une mauvaise chose et nous voulons l'éviter ! Il est de loin préférable de convertir tout notre texte en UTF-8 dès que possible et de le conserver dans cet encodage. Le meilleur moment pour convertir des données non UTF-8 en UTF-8 est lorsque vous lisez des fichiers, ce dont nous parlerons plus loin.

Cependant, essayez d'abord de convertir des octets et des chaînes de caractères avec des encodages différents et voyez ce qui se passe. Remarquez l'effet que cela a sur votre texte. Aimeriez-vous que cela se produise avec des données que vous essayez d'analyser ?

# Lecture de fichiers présentant des problèmes d'encodag

La plupart des fichiers que vous rencontrerez seront probablement encodés en UTF-8. C'est ce que Python attend par défaut, donc la plupart du temps vous ne rencontrerez pas de problèmes. Cependant, vous obtiendrez parfois une erreur comme celle-ci :

In [31]:
# essayer de lire un fichier qui n'est pas en UTF-8
kickstarter_2016 = pd.read_csv("/Users/jl/Downloads/ks-projects-201612.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 7955: invalid start byte

Remarquez que nous obtenons la même UnicodeDecodeError que lorsque nous avons essayé de décoder des octets UTF-8 comme s'il s'agissait d'ASCII ! Cela nous indique que ce fichier n'est pas réellement UTF-8. Nous ne savons pas quel est l'encodage réel. Une façon de le découvrir est d'essayer et de tester un tas d'encodages de caractères différents et de voir si l'un d'entre eux fonctionne. Une meilleure façon, cependant, est d'utiliser le module chardet pour essayer de deviner automatiquement quel est le bon encodage. Ce n'est pas garanti à 100%, mais c'est généralement plus rapide que d'essayer de deviner.

Je vais me contenter de regarder les dix mille premiers octets de ce fichier. C'est généralement suffisant pour avoir une bonne idée de l'encodage et c'est beaucoup plus rapide que d'essayer de regarder l'ensemble du fichier. (Une autre raison de ne regarder que la première partie du fichier est que nous pouvons voir, en regardant le message d'erreur, que le premier problème est le 11ème caractère. Nous n'avons donc probablement besoin que de la première partie du fichier pour comprendre ce qui se passe.

In [33]:
# regarder les dix mille premiers octets pour deviner l'encodage des caractères
with open("/Users/jl/Downloads/ks-projects-201801.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# vérifie l'encodage des caractères
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


Donc chardet est sûr à 73% que le bon encodage est "Windows-1252". Voyons si c'est correct :

In [34]:
# lire le fichier avec l'encodage détecté par chardet
kickstarter_2016 = pd.read_csv("/Users/jl/Downloads/ks-projects-201612.csv", encoding='Windows-1252')

# regarder les premières lignes
kickstarter_2016.head()

/var/folders/r5/sycktgpj0ln8vbwphhbh5mb00000gn/T/ipykernel_34170/13219038.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  kickstarter_2016 = pd.read_csv("/Users/jl/Downloads/ks-projects-201612.csv", encoding='Windows-1252')


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


Oui, on dirait que Chardet avait raison ! Le fichier se lit sans problème (bien que nous ayons un avertissement sur les types de données) et lorsque nous regardons les premières lignes, tout semble aller bien.

# Enregistrer vos fichiers avec le codage UTF-8

Enfin, une fois que vous vous êtes donné la peine de convertir votre fichier en UTF-8, vous voudrez probablement le conserver. La façon la plus simple de le faire est d'enregistrer vos fichiers avec l'encodage UTF-8. La bonne nouvelle est que, puisque l'UTF-8 est l'encodage standard de Python, lorsque vous enregistrez un fichier, il sera enregistré en UTF-8 par défaut :

In [35]:
# enregistrer notre fichier (il sera enregistré en UTF-8 par défaut !)
kickstarter_2016.to_csv("/Users/jl/Downloads/ks-projects-201801.csv")